In [1]:
import praw
import prawcore
import pandas as pd
import numpy as np 
import os
import re
# from itertools import chain, combinations, pairwise
from tqdm import tqdm
from datetime import datetime, date


# from spacy import load
# from spacy.tokenizer import Tokenizer
# from spacy.lang.en import English
import warnings
warnings.filterwarnings("ignore", message=r"\[W008\]", category=UserWarning)


In [2]:
path = 'data'
user_dir = 'user_data'
reddit = praw.Reddit(
    client_id='4bDjCY6y8ncrc3kLrBbpBg',
    client_secret='zPHZKfk9S666S9IxR5HtvkZ83ufNxw',
    user_agent='webcrawler created for IS596'
)
# nlp = load('en_core_web_md')
# titles = ['conservative_users.csv', 'conspiracy_users.csv', 'politics_users.csv', 'worldnews_users.csv']

Version 7.6.0 of praw is outdated. Version 7.6.1 was released Friday November 11, 2022.


In [3]:
def normalize(text):
    norm_text = []

    for token in text:
        if not token.is_punct and not token.is_stop and not token.is_space:
            norm_text.append(token.lemma_.lower())

    norm_text = ' '.join(norm_text)
    norm_text = re.sub(r'(?:^| )\w(?:$| )', ' ', norm_text).strip()  # removes single characters
    norm_text = re.sub(r'[^a-zA-Z0-9 ]', '', norm_text)
    
    return norm_text

In [4]:
def get_user_status(path):
    try:
        user.is_suspended
        user_status = 'suspended'
    except AttributeError:
        user_status = 'active'
    except:
        user_status = 'deleted'
    return user_status

In [5]:
def form_set(array):
    this_set = set(chain.from_iterable(i.split() for i in array))
    return this_set    

In [6]:
for filename in os.scandir('data'):
    if filename.is_file():
        
        file_df = pd.read_csv(filename)
        user_names = file_df.author
        users_df = pd.DataFrame(columns=['user_name', 'comment_simularity'])
        rows = []
        
        for user_name in tqdm(user_names[:5]):
            user_dict = {}
            user_dict['username'] = user_name
            user = reddit.redditor(user_name)
            
            if get_user_status(user) == 'active' and str(user_name) not in 'nan': # checks that user isn't suspended/deleted
                if not user.is_mod: #ignore mods
                    
                    comment_array = []
                    timestamps = []
                    reply_timestamps = []
                    
                    try:
                        for this_comment in user.comments.new(limit=10):
                            
                            parent_comment_id = this_comment.parent_id
                            if parent_comment_id.startswith('t3'):
                                parent = reddit.submission(parent_comment_id)
                            else:
                                parent = reddit.comment(parent_comment_id)
                            
                            parent_timestamp = datetime.fromtimestamp(parent.created_utc)
                            comment_timestamp = datetime.fromtimestamp(this_comment.created_utc)
                            
                            comment_array.append(this_comment.body)
                            timestamps.append(comment_timestamp)
                            reply_timestamps.append((parent_timestamp, comment_timestamp))
                            print('part1 success')
                    except Exception as e:
                        print(e)
                        continue
                        
                    for i in range(len(comment_array)):
                        comment_array[i] = nlp(normalize(nlp((comment_array[i])))) # normalizes comment

                    comment_similarities = []
                    time_intervals = []
                    response_intervals = []

                    for sent_1, sent_2 in combinations(comment_array, 2):
                        comment_similarities.append(sent_1.similarity(sent_2)) # checks cosine simularity of each comment against the next
                    
                    successive_times = list(pairwise(timestamps))
                    for pair in successive_times: # calculates the intervals between user's comments
                        time_intervals.append(abs(pair[0] - pair[1]))
                        
                    for pair in reply_times: 
                        response_intervals.append(abs(pair[1] - pair[0])) # calculates how quickly a comment replied to its parent
                        
                    time_data = pd.Series(time_intervals)
                    avg_time_diff = (time_data.sum()/len(time_data)).round('1s')
                    
                    response_data = pd.Series(response_intervals)
                    avg_reply_speed = (response_data.sum()/len(response_data)).round('1s')
                    
                    try:
                        avg_comment_similarity = sum(comment_similarities)/len(comment_similarities)
                    except ZeroDivisionError:
                        continue
                    except Exception as e:
                        print(e)
                        continue
                    
                    user_dict['avg_comment_similarity'] = avg_comment_similarity # formating dict
                    user_dict['avg_comment_time_interval'] = avg_time_diff
                    user_dict['avg_reply_speed'] = avg_reply_speed

                    rows.append(user_dict) 

        users_df = pd.DataFrame.from_dict(rows, orient='columns')
        print(users_df)
        
        filename = str(filename)
        
        if filename == "<DirEntry 'comment_conspiracy.csv'>":
            users_df.to_csv(os.path.join(f'{path}/{user_dir}', f'{titles[1]}'), index=False)
        if filename == "<DirEntry 'comment_politics.csv'>":
            users_df.to_csv(os.path.join(f'{path}/{user_dir}', f'{titles[2]}'), index=False)
        if filename == "<DirEntry 'submission_conservative.csv'>":
            users_df.to_csv(os.path.join(f'{path}/{user_dir}', f'{titles[0]}'))
        if filename == "<DirEntry 'submission_conspiracy.csv'>":
            users_df.to_csv(os.path.join(f'{path}/{user_dir}', f'{titles[1]}'), header=False, index=False, mode='a')
        if filename == "<DirEntry 'submission_politics.csv'>":
            users_df.to_csv(os.path.join(f'{path}/{user_dir}', f'{titles[2]}'), header=False, index=False, mode='a')
        if filename == "<DirEntry 'submission_worldnews.csv'>":
            users_df.to_csv(os.path.join(f'{path}/{user_dir}', f'{titles[3]}'))

 20%|██        | 1/5 [00:00<00:01,  2.41it/s]

received 404 HTTP response


 40%|████      | 2/5 [00:00<00:01,  2.15it/s]

part1 success
part1 success
received 404 HTTP response


 60%|██████    | 3/5 [00:01<00:00,  2.40it/s]

...

 60%|██████    | 3/5 [00:01<00:00,  2.80it/s]

received 404 HTTP response


100%|██████████| 5/5 [00:01<00:00,  3.33it/s]

received 404 HTTP response
Empty DataFrame
Columns: []
Index: []


NameError: name 'titles' is not defined

## Troubleshoot

In [7]:
user = reddit.redditor('readzalot1')
comment_array = []
timestamps = []
reply_times = []

for this_comment in user.comments.new(limit=5):
                            
    comment_timestamp = datetime.fromtimestamp(this_comment.created_utc)
    print(f'comment id: {this_comment.id}')
    print(f'comment timestamp: {comment_timestamp}')
    parent_comment_id = this_comment.parent_id
    
    print(this_comment.body)
    if parent_comment_id.startswith('t3'):
        print(f'parent_comment_id: {parent_comment_id}')
        parent = reddit.submission(parent_comment_id)
    else:
        print(f'parent_comment_id: {parent_comment_id}')
        parent = reddit.comment(parent_comment_id)
    try:
        parent_timestamp = datetime.fromtimestamp(parent.created_utc)
        print(f'parent timestamp: {parent_timestamp}')
        print()
    except Exception as e:
        print(e)
        print()
        continue
    comment_array.append(this_comment.body)
    timestamps.append(comment_timestamp)
    reply_times.append((parent_timestamp, comment_timestamp))

print(f'reply times:')
for reply in reply_times:
    print(reply)
response_intervals = []
for pair in reply_times:
    response_intervals.append(abs(pair[1] - pair[0]))

print()
print('response intervals:')
for response in response_intervals:
    print(response)

comment id: iyigroa
comment timestamp: 2022-12-01 11:23:20
Jan 6, no need to investigate. Hunter’s laptop tho
parent_comment_id: t3_z9l3h2
received 404 HTTP response

comment id: iyibc90
comment timestamp: 2022-12-01 10:48:45
He might have realized that his handlers are just pushing him around. Even intellectually disabled people are sensitive to that.
parent_comment_id: t1_iyi55w5
parent timestamp: 2022-12-01 10:07:20

comment id: iyht6hm
comment timestamp: 2022-12-01 08:43:18
But having to wait two hours in a red district is.
parent_comment_id: t1_iyhraex
parent timestamp: 2022-12-01 08:28:58

comment id: iygwsex
comment timestamp: 2022-12-01 02:26:50
I bought some tape that is like a tensor bandage that I wrap at the top of the boots. It holds them on better than anything else so far.
parent_comment_id: t1_iygex6u
parent timestamp: 2022-11-30 22:51:49

comment id: iygj4lc
comment timestamp: 2022-11-30 23:33:53
With Kenny, I was busy writing letters. With this one I too am just exhau

In [23]:
post = reddit.submission('z8pohf')

print(post.created_utc)

1669809570.0


In [9]:
reddit.comment('t1_iygex6u').created_utc

1669870309.0

In [12]:
parent_comment_id = 't3_z9l3h2'

In [16]:
parent = reddit.submission('t3_z9l3h2')
parent_timestamp = datetime.fromtimestamp(parent.created_utc)
print(f'parent timestamp: {parent_timestamp}')

NotFound: received 404 HTTP response

In [ ]:
parent_timestamp = datetime.fromtimestamp(parent.created_utc)
print(f'parent timestamp: {parent_timestamp}')

In [23]:
test = reddit.submission('t3_z9l3h2')

In [25]:
test.body

NotFound: received 404 HTTP response